In [ ]:
!%pip install -q ultralytics roboflow

In [ ]:
import os
from roboflow import Roboflow

# TODO: fill in your API key and project info
API_KEY = "YOUR_API_KEY"
WORKSPACE = "fpmoz"  # adjust if different
PROJECT = "brain-tumor-segmentation-jteuo-vwha2"
VERSION = 1

rf = Roboflow(api_key=API_KEY)
ds = rf.workspace(WORKSPACE).project(PROJECT).version(VERSION).download("yolov8")
data_yaml = os.path.join(ds.location, "data.yaml")
print("Dataset at:", ds.location)
print("data.yaml:", data_yaml)

import yaml
EXPECTED_NAMES = ["meningioma", "notumor"]
with open(data_yaml, "r") as f:
    data = yaml.safe_load(f)
data["names"] = EXPECTED_NAMES
data["nc"] = len(EXPECTED_NAMES)
with open(data_yaml, "w") as f:
    yaml.safe_dump(data, f)


In [ ]:
from ultralytics import YOLO

use_yolov11 = True
base_ckpt = "yolov8n-seg.pt" if use_yolov11 else "yolov8m-seg.pt"  

model = YOLO(base_ckpt)

model.train(
    data=data_yaml,
    imgsz=640,    
    epochs=100,   
    batch=8,      
    lr0=0.001,
    patience=40,  
    cache=True,
    device=0,     # GPU
)

best_path = model.trainer.best
print("Best checkpoint:", best_path)

In [ ]:
# Download best.pt to your machine
from google.colab import files
files.download(best_path)